In [1]:
#Importing the required libraries
import requests
from datetime import datetime, timedelta
import json
import random
import os

## Sales Order

In [22]:
#Function to call the API
def api_callSO(tranDate):
    response = requests.get(f'http://127.0.0.1:8000/soData?paramDate={tranDate}')
    data = response.json()        
    tupleToInsert = tuple(data.values())
    return tupleToInsert

#Function to create a SQL insert statement
def convert_to_sqlSO(data):
    sql_statements = []
    for record in data:
        values = ', '.join([f"'{str(value)}'" for value in record])
        sql_statement = f"""INSERT INTO salesorder
    (idCustomer, idItem, createdDate, dueDate, shipDate, qty, qtyFullfilled, qtyShipped, soStatus) 
    VALUES ({values});"""
        sql_statements.append(sql_statement)
    return sql_statements

#Function to store SQL script into a file
def store_sql_scriptSO(sql_statements, file_name='C:/Tesis/AnalyticsDocker/postgresql/soInsert.sql'):
    with open(file_name, 'w') as file:
        for statement in sql_statements:
            file.write(statement  + '\n')

In [23]:
#Starting date from the extraction
tranDate = datetime(2024, 1, 1).strftime("%Y-%m-%d")

#List that will contain the results
resultsSO = []

In [24]:
i = 0

#creting a iteration to retrieve the data and storing in into a dictionary
while True:
    #if its the first iter the starting date will be used
    if i == 0:
        resultsSO.append(api_callSO(tranDate))
    #for the following iter the last date will be used
    else:
        #this is used to get the last date and have only the date from it
        call = api_callSO(resultsSO[-1][2][:10])
        #only the year 2024 will be retrived
        if "2025" in call[2][:10]:
            break
        else:
            resultsSO.append(call)
    i += 1
    #security measuere
    if i == 10000:
        break

In [25]:
#creating the inserts statements
sql_statements_SO = convert_to_sqlSO(resultsSO)

In [26]:
#saving the statements
store_sql_scriptSO(sql_statements_SO)

## WO

In [27]:
#Function to call the API
def api_callWO(idSO, idItem, fromDate, qtyFullfilled, soStatus):
    response = requests.get(f'http://127.0.0.1:8000/woData?idSO={idSO}&idItem={idItem}&fromDate={fromDate}&qtyFullfilled={qtyFullfilled}&soStatus={soStatus}')
    data = response.json()        
    tupleToInsert = tuple(data.values())
    return tupleToInsert

#Function to create a SQL insert statement
def convert_to_sqlWO(data):
    sql_statements = []
    for record in data:
        values = ', '.join([f"'{str(value)}'" for value in record])
        sql_statement = f"""INSERT INTO workOrder 
    (idSO, idItem, createdDate, closedDate, qtyCreated, scrapQty) 
    VALUES ({values});"""
        sql_statements.append(sql_statement)
    return sql_statements

#Function to store SQL script into a file
def store_sql_scriptWO(sql_statements, file_name='C:/Tesis/AnalyticsDocker/postgresql/woInsert.sql'):
    with open(file_name, 'w') as file:
        for statement in sql_statements:
            file.write(statement  + '\n')

In [28]:
#List that will contain the results
resultsWO = []

In [29]:
#Iterating through the sales order to retrieve the working orders data
for i in range(len(resultsSO)):
    
    #this condition is needed in order to dont simulate wo data on orders that doesnt have stock related
    if resultsSO[i][8] == 'Approved':
        
        continue
    
    #because the id in the SO table is a Serial and we are inserting the data on the creation of the container
    #the position of the list will be used as the id because they are the same
    resultsWO.append(api_callWO(i+1, resultsSO[i][1], resultsSO[i][2][:10], resultsSO[i][6], resultsSO[i][8]))

In [30]:
#creating the inserts statements
sql_statements_WO = convert_to_sqlWO(resultsWO)

In [31]:
#saving the statements
store_sql_scriptWO(sql_statements_WO)

## Quota

In [32]:
#Function to call the API
def api_callQuota(fromDate, idItem):
    response = requests.get(f'http://127.0.0.1:8000/quota?fromDate={fromDate}&idItem={idItem}')
    data = response.json()        
    tupleToInsert = tuple(data.values())
    return tupleToInsert

#Function to create a SQL insert statement
def convert_to_sqlQuota(data):
    sql_statements = []
    for record in data:
        values = ', '.join([f"'{str(value)}'" for value in record])
        sql_statement = f"""INSERT INTO quota 
    (period, idItem, quota) 
    VALUES ({values});"""
        sql_statements.append(sql_statement)
    return sql_statements

#Function to store SQL script into a file
def store_sql_scriptQuota(sql_statements, file_name='C:/Tesis/AnalyticsDocker/postgresql/quotaInsert.sql'):
    with open(file_name, 'w') as file:
        for statement in sql_statements:
            file.write(statement  + '\n')

In [33]:
#List that will contain the results
resultsQuota = []

#List containing the periods for the Quota that needs to be generated
periods = ["2024-01-01",
           "2024-02-01",
           "2024-03-01",
           "2024-04-01",
           "2024-05-01",
           "2024-06-01",
           "2024-07-01",
           "2024-08-01",
           "2024-09-01",
           "2024-10-01",
           "2024-11-01",
           "2024-12-01"]

#List cointaing the ID of the items
items = ["1","2","3"]

In [34]:
for period in periods:
    for item in items:
        resultsQuota.append(api_callQuota(period, item))

In [35]:
#creating the inserts statements
sql_statements_Quota = convert_to_sqlQuota(resultsQuota)

In [36]:
#saving the statements
store_sql_scriptQuota(sql_statements_Quota)

## Sensor Data

In [15]:
#funtion to generate a list with the epoch beetween 2 dates, from 6 am to 2 pm with intervals of 2 minute
def generate_timestamps(start_date, end_date):
    current_date = start_date
    end_time = timedelta(hours=14)  # 2 PM
    timestamps_in_seconds = []

    while current_date <= end_date:
        time = timedelta(hours=6)  # 6 AM
        while time <= end_time:
            timestamp = datetime.combine(current_date, datetime.min.time()) + time
            timestamps_in_seconds.append(int(timestamp.timestamp()))
            time += timedelta(minutes=4)
        current_date += timedelta(days=1)

    return timestamps_in_seconds

#Function to create a SQL insert statement
def convert_to_sqlSensor(data):
    sql_statements = []
    for record in data:
        values = ', '.join([f"'{str(value)}'" for value in record])
        sql_statement = f"""INSERT INTO sensordata 
    (createdAt, mach, temp)
    VALUES ({values});"""
        sql_statements.append(sql_statement)
    return sql_statements

#Function to store SQL script into a file
def store_sql_scriptSensor(sql_statements, file_name='C:/Tesis/AnalyticsDocker/postgresql/sensorInsert.sql'):
    with open(file_name, 'w') as file:
        for statement in sql_statements:
            file.write(statement  + '\n')

In [16]:
#Creating a list with the days of the year 2024
start_date = datetime.strptime('2024-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2024-12-31', '%Y-%m-%d')

date_list = generate_timestamps(start_date, end_date)

In [17]:
#Defining the parameters wich the sensor data will be created
parameters = {
    '1': {'moldTemp': 75, 'tempDev': 10, 'maq': 'Iny 1'},
    '2': {'moldTemp': 40, 'tempDev': 5, 'maq': 'Iny 2'},
    '3': {'moldTemp': 90, 'tempDev': 18, 'maq': 'Iny 3'},
}

resultsSensor = []

In [18]:
for sensorDate in date_list:
    for parameter in parameters:
        mean   = parameters[parameter]['moldTemp']
        stdDev = parameters[parameter]['tempDev']
        resultsSensor.append(
                             (sensorDate, 
                              parameters[parameter]['maq'],
                              round(random.normalvariate(mean, stdDev),2))
                            )

In [19]:
#creating the inserts statements
sql_statements_Sensor = convert_to_sqlSensor(resultsSensor)

In [20]:
#saving the statements
store_sql_scriptSensor(sql_statements_Sensor)

## Files Concatenation

In [37]:
#Functions

def concatenate_sql_files(file_list, input_folder, output_file):
    with open(output_file, 'w') as outfile:
        for filename in file_list:
            filepath = os.path.join(input_folder, filename)
            with open(filepath, 'r') as infile:
                # Write the content of each file to the output file
                content = infile.read()
                content = content.replace("'None'", 'NULL')
                outfile.write(content)
                outfile.write("\n")  # Add a newline for separation between files

def get_sql_file_list(input_folder):
    # Get the list of files in the input folder
    files = [f for f in os.listdir(input_folder) if f.endswith('.sql')]
    
    # Sort the files by name
    files.sort()
    
    return files

In [39]:
file_list = get_sql_file_list('C:/Tesis/AnalyticsDocker/postgresql/')

print(file_list)

['quotaInsert.sql', 'sensorInsert.sql', 'soInsert.sql', 'sqlCreateTable.sql', 'sqlCreateViews.sql', 'sqlInit.sql', 'sqlInsertDimensionData.sql', 'sqlReset.sql', 'woInsert.sql']


In [40]:
#defining the order wich the files will be concatenated
file_list_ordered = ['sqlCreateTable.sql',
                     'sqlInsertDimensionData.sql',
                     'soInsert.sql',
                     'woInsert.sql',
                     'quotaInsert.sql',
                     'sensorInsert.sql',
                     'sqlCreateViews.sql', 
                     'sqlReset.sql',
                     ]

In [41]:
#concatenating the files
concatenate_sql_files(file_list_ordered, 'C:/Tesis/AnalyticsDocker/postgresql', 'C:/Tesis/AnalyticsDocker/postgresql/sqlInit.sql')